In [1]:
import load_env

In [2]:
import os
import pathlib
import re

import pandas as pd
import supabase

import supabasefs
from scanner import Scanner

from stats_collector import StatsCollector

2023-05-22 20:58:24,240:INFO - Using TgCrypto


In [3]:
client = supabase.create_client(
    os.environ["SUPABASE_URL"], os.environ["SUPABASE_KEY"]
)
fs = supabasefs.SupabaseTableFileSystem(client, "sessions")
scanner = Scanner(fs=fs, chat_cache=False)
collector = StatsCollector(scanner)

In [4]:
CHANNEL = "@flipping_invest"

async with scanner.session():
    results = await collector.collect_msg_stats(CHANNEL)

df = pd.DataFrame(results)

2023-05-22 20:58:31,369:INFO - Connecting...
2023-05-22 20:58:32,888:INFO - Connecting...
2023-05-22 20:58:34,837:INFO - Connecting...
2023-05-22 20:58:36,410:INFO - Connecting...
2023-05-22 20:58:36,458:INFO - Connected! Production DC2 - IPv4
2023-05-22 20:58:36,460:INFO - NetworkTask started
2023-05-22 20:58:36,463:INFO - Connected! Production DC2 - IPv4
2023-05-22 20:58:36,468:INFO - Connected! Production DC2 - IPv4
2023-05-22 20:58:36,470:INFO - Connected! Production DC2 - IPv4
2023-05-22 20:58:36,471:INFO - NetworkTask started
2023-05-22 20:58:36,472:INFO - NetworkTask started
2023-05-22 20:58:36,474:INFO - NetworkTask started
2023-05-22 20:58:36,809:INFO - Session initialized: Layer 158
2023-05-22 20:58:36,810:INFO - Device: CPython 3.11.3 - Pyrogram 2.0.104
2023-05-22 20:58:36,811:INFO - System: Windows 10 (en)
2023-05-22 20:58:36,813:INFO - Session started
2023-05-22 20:58:36,876:INFO - PingTask started
2023-05-22 20:58:36,882:INFO - Session initialized: Layer 158
2023-05-22 20

In [5]:
df["popularity"] = df.reactions / df.reach * 100

In [6]:
top = df[df.datetime < pd.Timestamp.now() - pd.DateOffset(days=3)].sort_values("popularity", ascending=False).head(20)
for col in ["reach", "reactions"]:
    top[col] = pd.to_numeric(top[col], errors="ignore", downcast="integer")
top

,username,link,reach,reactions,datetime,text,popularity
222,@flipping_invest,https://t.me/flipping_invest/46,1262,117,2022-09-28 13:48:07,БЕГ?\n\nОчень сложно было в последнюю неделю. ...,9.270998
51,@flipping_invest,https://t.me/flipping_invest/241,1591,146,2023-04-17 08:20:31,(7/10) Как купить компанию за 10% от стоимости...,9.176618
109,@flipping_invest,https://t.me/flipping_invest/173,2300,197,2023-01-29 18:50:19,12 МЛН. ДАРОМ!\n(Ну почти :))\n\nНехитрая схем...,8.565217
36,@flipping_invest,https://t.me/flipping_invest/259,1180,76,2023-04-28 09:52:58,(10/10) Налоги: оптимизировать нельзя платить\...,6.440678
16,@flipping_invest,https://t.me/flipping_invest/282,1293,81,2023-05-11 13:31:33,Как мы из сарая хотели сделать гостиницу\n\n(6...,6.264501
52,@flipping_invest,https://t.me/flipping_invest/240,1381,86,2023-04-16 11:35:59,None,6.227371
234,@flipping_invest,https://t.me/flipping_invest/32,957,53,2022-08-26 11:21:33,Решил чуть-чуть разбавить сухой контент в этом...,5.538140
150,@flipping_invest,https://t.me/flipping_invest/128,1717,95,2022-12-15 19:51:36,"Асхн тёвкнюн болтха!\n\nОй, простите, это по-к...",5.532906
138,@flipping_invest,https://t.me/flipping_invest/141,1640,89,2022-12-29 00:30:00,#снято\n\nФух! Сегодня закончили выкуп гостини...,5.426829
60,@flipping_invest,https://t.me/flipping_invest/232,1639,85,2023-04-11 08:31:02,(6/10) Коттеджный поселок в Подмосковье: почем...,5.186089


In [7]:
html = top.to_html(encoding="utf-8")
html = re.sub(r'<td>(https://.+)</td>', r'<td><a href="\g<1>">ссылка</a></td>', html)
pathlib.Path("top.html").write_text(html, encoding="utf-8")

ValueError: buf is not a file name and encoding is specified.